In [1]:
import pandas as pd
import numpy as np

from ethanol_brazil_plants.utils import utils

from datetime import date
from xlrd.xldate import xldate_as_datetime

In [2]:
raw_rbp_validos = catalog.load("raw_renovabio_plants_validos")

                    INFO     Loading data from 'raw_renovabio_plants_validos' (ExcelDataSet)... ]8;id=261835;file://C:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=581850;file://C:\Users\daniel.silva\AppData\Local\miniconda3\envs\kedro-std\lib\site-packages\kedro\io\data_catalog.py#475\475]8;;\

In [3]:
def generate_razao_cidade_uf(df_rbp_validos):
    """
    Generate RAZAO_SOCIAL, CIDADE and UF columns.
    """

    df_rbp_validos['RAZAO_SOCIAL'] = df_rbp_validos['RAZAO_SOCIAL-CIDADE-UF'].str.split('-').str[0]

    df_rbp_validos[['CIDADE', 'UF', 'UF_2']] = (
        df_rbp_validos['RAZAO_SOCIAL-CIDADE-UF']
        .str.split('-')
        .str[-2:]
        .str.join(',')
        .str.split(',', expand=True)
    )

    df_rbp_validos['UF'] = np.where(
        df_rbp_validos['UF'].isnull(),
        df_rbp_validos['UF_2'],
        df_rbp_validos['UF']
    )

    df_rbp_validos['CIDADE'] = np.where(
        df_rbp_validos['RAZAO_SOCIAL'] !=
        df_rbp_validos['CIDADE'],
        df_rbp_validos['CIDADE'],
        None
    )

    df_rbp_validos = df_rbp_validos.drop(['RAZAO_SOCIAL-CIDADE-UF', 'UF_2'], axis=1)
    return df_rbp_validos

In [ ]:
# Normalize
to_normalize = [
    'RAZAO_SOCIAL-CIDADE-UF',
    'BIOCOMBUSTIVEL',
    'ROTA',
    'FIRMA_INSPETORA',
    'ENDERECO_EMISSOR_PRIMARIO',
    'CNPJ',
    'PROCESSO_DE_CERTIFICACAO',
]

to_int = [
    'CNPJ',
    'PROCESSO_DE_CERTIFICACAO']
    
to_float = [
    'NOTA_DE_EFICIENCIA_ENERGETICO-AMBIENTAL_GCO2EQ/MJ',
    'VOLUME_ELEGIVEL_%',
    'FATOR_PARA_EMISSAO_DE_CBIO_TCO2EQ/L',
]

date_cols = ['DATA_DE_APROVACAO_PELA_ANP', 'VALIDADE']

df_rbp_validos = utils.normalize_header(raw_rbp_validos)
df_rbp_validos = df_rbp_validos.fillna(method='ffill')

df_rbp_validos = utils.normalize_object_columns(df_rbp_validos, to_normalize)
df_rbp_validos = df_rbp_validos[df_rbp_validos['CNPJ'] != '']

df_rbp_validos = utils.normalize_int_columns(df_rbp_validos, to_int)
df_rbp_validos = utils.normalize_float_columns(df_rbp_validos, to_float)

df_rbp_validos = utils.normalize_date_columns(df_rbp_validos, date_cols)


In [256]:
df_rbp_validos['RAZAO_SOCIAL'] = df_rbp_validos['RAZAO_SOCIAL-CIDADE-UF'].str.split('-').str[0]

df_rbp_validos[['CIDADE', 'UF', 'UF_2']] = (
    df_rbp_validos['RAZAO_SOCIAL-CIDADE-UF']
    .str.split('-')
    .str[-2:]
    .str.join(',')
    .str.split(',', expand=True)
)

In [257]:
df_rbp_validos['UF'] = np.where(
    df_rbp_validos['UF'].isnull(),
    df_rbp_validos['UF_2'],
    df_rbp_validos['UF']
)

df_rbp_validos['CIDADE'] = np.where(
    df_rbp_validos['RAZAO_SOCIAL'] !=
    df_rbp_validos['CIDADE'],
    df_rbp_validos['CIDADE'],
    None
)

df_rbp_validos = df_rbp_validos.drop(['RAZAO_SOCIAL-CIDADE-UF', 'UF_2'], axis=1)

In [258]:
rota_dict = {
    'ETANOL COMBUSTIVEL DE PRIMEIRA GERACAO CANA-DE-ACUCAR':
    'ETANOL 1GEN CANA',
    'ETANOL COMBUSTIVEL DE PRIMEIRA GERACAO CANA-DE-ACUCAR E MILHO EM USINA INTEGRADA':
    'ETANOL 1GEN CANA E MILHO',
    'ETANOL COMBUSTIVEL DE PRIMEIRA GERACAO MILHO':
    'ETANOL 1GEN MILHO',
    'ETANOL COMBUSTIVEL DE PRIMEIRA E SEGUNDA GERACAO PRODUZIDO EM USINA INTEGRADA':
    'ETANOL 1GEN E 2GEN',
}

df_rbp_validos['ROTA'] = df_rbp_validos['ROTA'].replace(rota_dict)

In [261]:
df_rbp_validos

,CNPJ,PROCESSO_DE_CERTIFICACAO,BIOCOMBUSTIVEL,ROTA,NOTA_DE_EFICIENCIA_ENERGETICO-AMBIENTAL_GCO2EQ/MJ,VOLUME_ELEGIVEL_%,FATOR_PARA_EMISSAO_DE_CBIO_TCO2EQ/L,LITROS/CBIO,DATA_DE_APROVACAO_PELA_ANP,VALIDADE,FIRMA_INSPETORA,ENDERECO_EMISSOR_PRIMARIO,RAZAO_SOCIAL,CIDADE,UF
0,44836856000177,48610206100202048,ETANOL HIDRATADO,ETANOL 1GEN CANA,53.40,93.90,0.001070,934.479878,2020-09-16 00:00:00,2023-09-15 00:00:00,BENRI CLASSIFICACAO DA PRODUCAO DE ACUCAR E ET...,"FAZENDA VISTA ALEGRE S/N PINHAL, ITAPETININGA ...",AGROINDUSTRIAL VISTA ALEGRE S. A. EM RECUPERAC...,ITAPETININGA,SP
1,44836856000177,48610206100202048,ETANOL ANIDRO,ETANOL 1GEN CANA,53.80,93.90,0.001129,885.530171,2020-09-16 00:00:00,2023-09-15 00:00:00,BENRI CLASSIFICACAO DA PRODUCAO DE ACUCAR E ET...,"FAZENDA VISTA ALEGRE S/N PINHAL, ITAPETININGA ...",AGROINDUSTRIAL VISTA ALEGRE S. A. EM RECUPERAC...,ITAPETININGA,SP
2,738822000255,48610200428202051,ETANOL HIDRATADO,ETANOL 1GEN CANA,53.40,100.00,0.001140,877.476238,2020-09-16 00:00:00,2023-09-15 00:00:00,SGS ICS CERTIFICADORA LTDA.,FAZENDA SANTA CLARA S/N - CEP: 45.807-000 - SA...,SANTA CRUZ ACUCAR E ALCOOL LTDA.,SANTA CRUZ CABRALIA,BA
3,738822000255,48610200428202051,ETANOL ANIDRO,ETANOL 1GEN CANA,53.80,100.00,0.001203,831.513013,2020-09-16 00:00:00,2023-09-15 00:00:00,SGS ICS CERTIFICADORA LTDA.,FAZENDA SANTA CLARA S/N - CEP: 45.807-000 - SA...,SANTA CRUZ ACUCAR E ALCOOL LTDA.,SANTA CRUZ CABRALIA,BA
4,28620879000193,48610219471201956,ETANOL HIDRATADO,ETANOL 1GEN CANA,48.10,86.75,0.000891,1122.954412,2020-09-24 00:00:00,2023-09-23 00:00:00,INSTITUTO TOTUM DE DESENVOLVIMENTO E GESTAO EM...,"FAZENDA SAO MATHEUS, S/N, ZONA RURAL, 57265-00...",IMPACTO BIOENERGIA ALAGOAS S.A.,TEOTONIO VILELA,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,5158542000100,48610217724202333,ETANOL HIDRATADO,ETANOL 1GEN CANA,59.67,29.17,0.000371,2692.056710,2023-07-25 00:00:00,2026-07-24 00:00:00,BENRI CLASSIFICACAO DA PRODUCAO DE ACUCAR E ET...,"FAZENDA ENGENHO NOVO, S/N. ZONA RURAL. PORTO C...",CENTRAL ACUCAREIRA USINA SANTA MARIA S/A.,None,None
503,5158542000100,48610217724202333,ETANOL ANIDRO,ETANOL 1GEN CANA,60.02,29.17,0.000391,2555.164729,2023-07-25 00:00:00,2026-07-24 00:00:00,BENRI CLASSIFICACAO DA PRODUCAO DE ACUCAR E ET...,"FAZENDA ENGENHO NOVO, S/N. ZONA RURAL. PORTO C...",CENTRAL ACUCAREIRA USINA SANTA MARIA S/A.,None,None
504,9094632000217,48610210276202347,ETANOL HIDRATADO,ETANOL 1GEN CANA,54.02,72.88,0.000840,1190.183037,2023-07-25 00:00:00,2026-07-24 00:00:00,GREEN DOMUS DESENVOLVIMENTO SUSTENTAVEL LTDA EPP,"FAZENDA MONTE ALEGRE , S/N ZONA RUARAL - MAMAG...",USINA MONTE ALEGRE AS,None,None
505,9094632000217,48610210276202347,ETANOL ANIDRO,ETANOL 1GEN CANA,54.37,72.88,0.000886,1128.973224,2023-07-25 00:00:00,2026-07-24 00:00:00,GREEN DOMUS DESENVOLVIMENTO SUSTENTAVEL LTDA EPP,"FAZENDA MONTE ALEGRE , S/N ZONA RUARAL - MAMAG...",USINA MONTE ALEGRE AS,None,None
